In [0]:
import h5py
import tensorflow as tf
tf.keras.backend.set_floatx('float32')

class lstm_model():
  def __init__(self, lstm_units, timesteps_in, timesteps_out, num_features, directory):
    self.lstm_units = lstm_units
    self.dropout_rate = 0.2
    self.timesteps_in = timesteps_in
    self.timesteps_out = timesteps_out
    self.num_features = num_features
    self.weights_dir = directory + "lstm.h5py"
    self.epochs = 20
    self.batch_size = 64
    self.create_model()

  def create_model(self):
    input_ = tf.keras.Input(shape=(self.timesteps_in, self.num_features))
    x = tf.keras.layers.LSTM(self.lstm_units, 
                             return_sequences=True, 
                             return_state=False)(input_)
    #x = tf.keras.layers.Dropout(self.dropout_rate)       
    x = tf.keras.layers.LSTM(self.lstm_units, 
                             return_sequences=False, 
                             return_state=False)(x)                             
    output = tf.keras.layers.Dense(self.timesteps_out, activation = "linear")(x)
    self.model = tf.keras.Model(inputs=input_, outputs=output)
    self.model.compile(loss="mae", optimizer="adam")
    self.model.summary()
  
  def fit_model(self, x_train, y_train, x_valid, y_valid):
    modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(self.weights_dir, 
                                                         monitor='val_loss', 
                                                         verbose=0, 
                                                         save_best_only=True,
                                                         save_weights_only=True)
    earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks = [modelcheckpoint, earlystopping]
    self.history = self.model.fit(x_train, y_train, batch_size=self.batch_size, 
                                  epochs=self.epochs, verbose=1, callbacks=callbacks, 
                                  validation_data=(x_valid, y_valid))

  def predict(self, x_test):
    self.model.load_weights(self.weights_dir)
    y_pred = self.model.predict(x_test, batch_size=self.batch_size)
    return y_pred